## Импорт и инициализация

In [4]:
import requests
import psycopg2
import json
import csv
import re
import pandas as pd
import html
import time
import numpy as np
import gensim.downloader as api
import nltk
import string
from geopy.geocoders import Nominatim
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Tuple, Dict
from datetime import datetime
from bs4 import BeautifulSoup

In [2]:
connection = psycopg2.connect(
    host='localhost',
    port=5433,
    user='postgres',
    password='1029384756',
    dbname='vacancy_analysis'
)
cursor = connection.cursor()

currency_TOKEN = "928dce37a9934ff99994af3d811c448e"
currency_values = requests.get(f"https://api.currencyfreaks.com/v2.0/rates/latest?apikey={currency_TOKEN}").json()["rates"]

loc = Nominatim(user_agent="GetLoc")

def convert_with_checking(func, dict, target):
    '''
        Функция конертации с проверкой на наличие поля в json
    '''
    if dict:
        value = dict[target]
        if value:
            return func(value)
        return None
    return None


def convert_to_RUB(value, from_currency):
    RUB_value = float(currency_values["RUB"])
    from_currency_value = float(currency_values[from_currency])
    to_USD = value / from_currency_value
    return int(to_USD * RUB_value)


def get_proper_skill_getmatch(skill_tag):
    result = []
    splitted = [word.strip() for word in skill_tag.split("/")]
    i = 0
    changed = False
    while i != len(splitted):
        if splitted[i].isdigit():
            if not(changed) and result:
                result[-1] = re.sub(r'\d+$', '', result[-1]).strip()
                changed = True
        else:
            if len(splitted[i]) <= 45:
                result.append(splitted[i])
                changed = False
        i += 1
    return result


def extract_requirements_segment(html: str) -> str:
    soup = BeautifulSoup(html, 'html.parser')

    # Удалим скрипты, стили и ненужное
    for tag in soup(['script', 'style']):
        tag.decompose()

    text = soup.get_text(separator='\n')
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    # Ключевые маркеры начала требований
    requirement_headers = [
        r'Требования',
        r'Что мы ожидаем',
        r'Что ты умеешь',
        r'Нам нужен',
        r'Подойдет кандидат',
        r'Ожидания',
        r'Что нам важно',
        r'Технические навыки',
        r'Ваш опыт'
    ]
    pattern = re.compile('|'.join(requirement_headers))

    start_idx = None
    end_idx = None

    # Найдём начало сегмента
    for i, line in enumerate(lines):
        if pattern.search(line):
            start_idx = i
            break

    if start_idx is not None:
        # Ограничим до следующего заголовка (условия, задачи, мы предлагаем и т.п.)
        end_headers = [
            r'Обязанности',
            r'Что делать',
            r'Условия',
            r'Предлагаем',
            r'Компания',
            r'О нас',
            r'Контакты',
            r'Офис',
            r'Зарплата',
            r'Преимущества'
        ]
        end_pattern = re.compile('|'.join(end_headers))

        for j in range(start_idx + 1, len(lines)):
            if end_pattern.search(lines[j]):
                end_idx = j
                break

        selected = lines[start_idx:end_idx] if end_idx else lines[start_idx:]
        return ' '.join(selected)
    
    return ' '.join(lines[5:30])

## Парсинг технических навыков

In [3]:
cursor.execute("select id, name from skills")
tech_skills = dict([(name, id) for id, name in cursor.fetchall()])
skills = set(tech_skills.keys())
parsing_skills = set([])

In [15]:
# Парсинг Habr career
i = 1
while True:
    habr_response = requests.get(f"https://career.habr.com/vacancies?page={i}&s[]=22&s[]=17&s[]=18&s[]=183&s[]=19&s[]=187&s[]=20&s[]=89&s[]=108&s[]=129&s[]=130&s[]=51&s[]=52&s[]=53&s[]=102&s[]=103&s[]=104&s[]=120&s[]=121&s[]=113&s[]=132&s[]=131&s[]=179&s[]=49&s[]=45&s[]=46&s[]=50&s[]=47&s[]=48&s[]=101&s[]=112&s[]=44&s[]=125&s[]=177&s[]=175&s[]=126&s[]=78&s[]=21&s[]=172&s[]=174&s[]=79&s[]=173&s[]=80&s[]=176&s[]=81&s[]=118&s[]=182&s[]=32&s[]=33&s[]=34&s[]=119&s[]=185&s[]=36&s[]=186&s[]=37&s[]=110&s[]=94&s[]=23&s[]=24&s[]=30&s[]=25&s[]=27&s[]=26&s[]=90&s[]=28&s[]=91&s[]=92&s[]=29&s[]=93&s[]=122&s[]=31&s[]=109&s[]=98&s[]=41&s[]=42&s[]=43&s[]=168&s[]=99&s[]=76&s[]=96&s[]=97&s[]=95&s[]=100&s[]=133&s[]=111&s[]=12&s[]=10&s[]=13&s[]=87&s[]=11&s[]=14&s[]=15&s[]=16&s[]=107&s[]=2&s[]=3&s[]=4&s[]=82&s[]=72&s[]=5&s[]=75&s[]=6&s[]=1&s[]=77&s[]=7&s[]=83&s[]=84&s[]=73&s[]=8&s[]=85&s[]=86&s[]=188&s[]=178&s[]=106&type=all")
    soup = BeautifulSoup(habr_response.text, 'html.parser')
    tags = soup.select('.vacancy-card__skills a')
    if not tags:
        break
    for tag in tags:
        parsing_skills.add(tag.text.strip().lower())
    i += 1
    if i % 10 == 0:
        print(i)
print("all done")

10
20
30
40
50
60
70
all done


In [17]:
# Парсинг getmatch
response = requests.get("https://getmatch.ru/vacancies?p=1&sa=150000&pa=all&s=landing_ca_vacancies")
soup = BeautifulSoup(response.content, 'html.parser')

pages = []
for page_num in soup.find_all(class_='b-pagination-page ng-star-inserted'):
    if (num := page_num.text.strip()).isdigit(): pages.append(int(num))
max_page = max(pages)

for page in range(1, max_page+1):
    response = requests.get(f"https://getmatch.ru/vacancies?p={page}&sa=150000&pa=all&s=landing_ca_vacancies")
    soup = BeautifulSoup(response.content, 'html.parser')
    for skill_tag in soup.select('div.b-vacancy-card-subtitle__stack span'):
        if (lowered_tag := skill_tag.text.strip().lower()) in skills: continue
        for skill in get_proper_skill_getmatch(re.sub(r"\([\w\W]+\)", '', lowered_tag)):
            parsing_skills.add(skill.strip())
    if page % 10 == 0:
        print(f"{page} done")
print("all done")

10 done
20 done
30 done
40 done
50 done
all done


## Обработка

In [4]:
cursor.execute("SELECT * FROM roles")
roles = dict([(row[0], row[1]) for row in cursor.fetchall()])
all_roles_id = roles.keys()

In [ ]:
all_vacancies = []
try:
    for id in all_roles_id:
        vacancies = []
        i = 0
        response = requests.get(f"https://api.hh.ru/vacancies?area=113&per_page=100&page={i}&order_by=publication_time&ored_clusters=true&professional_role={id}&period=1")
        response_data = response.json()
        print(f"{roles[id]}: {response.status_code}, {response.reason}, {response_data['found']}")
        vacancies += response_data["items"]
        if to := response_data["found"] // 100:
            for i in range(1, to + 1):
                response = requests.get(f"https://api.hh.ru/vacancies?area=113&per_page=100&page={i}&order_by=publication_time&ored_clusters=true&professional_role={id}&period=1").json()
                vacancies += response["items"]
        all_vacancies.append({"role_id": id, "vacancies": vacancies})
except Exception as e:
    print(e)

In [ ]:
to_dump = {"items": all_vacancies}
with open("json_results/data.json", 'w', encoding='utf-16') as file:
    json.dump(to_dump, file)

In [5]:
file = open("json_results/data.json", 'r', encoding='utf-16').read()
data = json.loads(file)

In [6]:
vacancy_data = []
vacancy_role = []
vacancy_work_formats = []
employers = {}
i = 0
for role_data in data["items"]:
    for vacancy in role_data["vacancies"]:
        if not("id" in vacancy["employer"]): continue
        
        salary_from = convert_with_checking(int, vacancy["salary_range"], "from")
        salary_to = convert_with_checking(int, vacancy["salary_range"], "to")

        if vacancy["salary_range"] and (currency := vacancy["salary_range"]["currency"]) != "RUR":
            if salary_from: salary_from = convert_to_RUB(salary_from, currency)
            if salary_to: salary_to = convert_to_RUB(salary_to, currency)
        
        vacancy_data.append([
            int(vacancy["id"]),
            vacancy["area"]["name"],
            convert_with_checking(float, vacancy["address"], "lat"),
            convert_with_checking(float, vacancy["address"], "lng"),
            vacancy["archived"],
            datetime.fromisoformat(vacancy["created_at"]),
            datetime.fromisoformat(vacancy["published_at"]),
            vacancy["has_test"],
            vacancy["internship"],
            salary_from,
            salary_to,
            vacancy["salary_range"]["frequency"]["name"].replace('\xa0', ' ') if vacancy["salary_range"] and vacancy["salary_range"]["frequency"] else "Неизвестно",
            int(vacancy["employer"]["id"]),
            vacancy["experience"]["name"],
            vacancy["employment"]["name"]
        ])
        
        vacancy_role.append([
            int(vacancy["id"]),
            role_data["role_id"]
        ])
        
        vacancy_work_formats += [
            [int(vacancy["id"]), work_format["name"].replace("\xa0", " ")]
            for work_format in vacancy["work_format"]
        ]
                    
        employer = employers.get(vacancy["employer"]["id"], [])
        if not employer:
            employers[int(vacancy["employer"]["id"])] = [
                vacancy["employer"]["name"],
                None if not("employer_rating" in vacancy["employer"]) else convert_with_checking(float, vacancy["employer"]["employer_rating"], "total_rating"),
                None if not("employer_rating" in vacancy["employer"]) else convert_with_checking(int, vacancy["employer"]["employer_rating"], "reviews_count"),
                vacancy["employer"]["accredited_it_employer"],
                vacancy["employer"]["trusted"],
                vacancy["employer"]["logo_urls"]["original"] if vacancy["employer"]["logo_urls"] else None
            ]
        else:
            rating = None if not("employer_rating" in vacancy["employer"]) else convert_with_checking(float, vacancy["employer"]["employer_rating"], "total_rating")
            reviews = None if not("employer_rating" in vacancy["employer"]) else convert_with_checking(int, vacancy["employer"]["employer_rating"], "reviews_count")
            log_url = vacancy["employer"]["logo_urls"]["original"] if vacancy["employer"]["logo_urls"] else None
            if not employer[1] and rating:
                employer[1] = rating
            if not employer[2] and reviews:
                employer[2] = reviews
            if not employer[5] and log_url:
                employer[5] = log_url
                    
employers = [[key] + value for key, value in employers.items()]

print("Done")

Done


In [7]:
vacancies_column_names = [
    'id',
    'area',
    'latitude',
    'longitude',
    'archived',
    'created_at',
    'published_at',
    'has_test',
    'internship',
    'salary_from',
    'salary_to',
    'salary_frequency',
    'company_id',
    'experience',
    'employment'
]
employers_column_names = [
    'id',
    'name',
    'total_rating',
    'reviews_count',
    'accredited_it_employer',
    'trusted',
    'logo_url'
]
vacancy_roles_column_names = [
    'vacancy_id',
    'role_id'
]
vacancy_work_formats_column_names = [
    'vacancy_id',
    'work_format'
]


vacancies_df = pd.DataFrame(vacancy_data, columns=vacancies_column_names)
vacancies_df.drop_duplicates(subset=["id"], keep="first", inplace=True)

employers_df = pd.DataFrame(employers, columns=employers_column_names)

vacancy_roles_df = pd.DataFrame(vacancy_role, columns=vacancy_roles_column_names)
vacancy_roles_df.drop_duplicates(keep="first", inplace=True)

vacancy_work_formats_df = pd.DataFrame(vacancy_work_formats, columns=vacancy_work_formats_column_names)
vacancy_work_formats_df.drop_duplicates(keep="first", inplace=True)

print("Done")

Done


In [8]:
print("[#] Обработка null значений с вакансии")
print("[#] До обработки")

print(f"[-] Общее количество записей: {vacancies_df['id'].count()}")
print("[-] Количество null значений в vacancies_df:")
print(vacancies_df.isnull().sum())

nulls_id = vacancies_df[
    vacancies_df['salary_from'].isnull() | (vacancies_df['salary_from'].isnull() & vacancies_df['salary_to'].isnull())
]["id"].copy()
print(f"[-] Количество null значений с salary_from или salary_from & salary_to: {len(nulls_id.index)}")
print("[&] Удаляем вакансии с salary_from == null или salary_from & salary_to == null...")
vacancies_df.drop(nulls_id.index, inplace=True)

print("[&] Заполняем долготу и широту по названию региона...")
areas_to_fix = vacancies_df[vacancies_df['longitude'].isnull()]["area"].unique()

cursor.execute("SELECT name, latitude, longitude FROM area_coordinates")

coordinates = {area[0]: [float(area[1]), float(area[2])] for area in cursor.fetchall()}
new_coordinates = {}
for area in areas_to_fix:
    if not(coordinates.get(area, [])):
        getLoc = loc.geocode(area)
        if getLoc:
            latitude = getLoc.latitude
            longitude = getLoc.longitude
            coordinates[area] = [latitude, longitude]
            new_coordinates[area] = [latitude, longitude]

if new_coordinates:
    with open("add_new_coordinates.sql", "w") as file:
        file.write("INSERT INTO area_coordinates (name, latitude, longitude) VALUES\n")
        file.write(
            ",\n".join(
                [
                    f"('{name}', {latitude}, {longtitude})"
                    for (name, (latitude, longtitude)) in new_coordinates.items()
                ]
            ) + ";"
        )

have_null_areas = False

if len(coordinates.keys()) == len(areas_to_fix):
    print("[&] Есть все координаты, меняем null...")
else:
    print(f"[-] Количество ненайденных местностей: {len(areas_to_fix) - len(coordinates)}")
    print(f"[-] Сбрасываем их.", end=" ")
    have_null_areas = True
    null_areas = vacancies_df[
        ~vacancies_df['area'].isin(coordinates.keys())
    ]["id"].copy()
    vacancies_df.drop(null_areas.index, inplace=True)
    print(f"Количество сброшенных строк: {len(null_areas.index)}")
    print("[&] Меняем null...")

vacancies_df.loc[:, ['latitude', 'longitude']] = vacancies_df["area"].apply(lambda x: coordinates[x]).values.tolist()

print("[#] После обработки")
print(f"[-] Общее количество записей: {vacancies_df["id"].count()}")
print(vacancies_df.isnull().sum())

nulls_id = sorted(nulls_id.values.tolist() + null_areas.values.tolist()) if have_null_areas else sorted(nulls_id.values.tolist())

[#] Обработка null значений с вакансии
[#] До обработки
[-] Общее количество записей: 6088
[-] Количество null значений в vacancies_df:
id                     0
area                   0
latitude            2259
longitude           2259
archived               0
created_at             0
published_at           0
has_test               0
internship             0
salary_from         3370
salary_to           4428
salary_frequency       0
company_id             0
experience             0
employment             0
dtype: int64
[-] Количество null значений с salary_from или salary_from & salary_to: 3370
[&] Удаляем вакансии с salary_from == null или salary_from & salary_to == null...
[&] Заполняем долготу и широту по названию региона...
[-] Количество ненайденных местностей: 1
[-] Сбрасываем их. Количество сброшенных строк: 138
[&] Меняем null...
[#] После обработки
[-] Общее количество записей: 2580
id                     0
area                   0
latitude               0
longitude            

In [9]:
# Удалим данные связанные с удаленными вакансиями
dropping_vacancy_roles_index = vacancy_roles_df[vacancy_roles_df["vacancy_id"].isin(nulls_id)].index
vacancy_roles_df.drop(dropping_vacancy_roles_index, inplace=True)

dropping_vacancy_work_formats_index = vacancy_work_formats_df[vacancy_work_formats_df["vacancy_id"].isin(nulls_id)].index
vacancy_work_formats_df.drop(dropping_vacancy_work_formats_index, inplace=True)
print("Done")

Done


### Выделение технических навыков

In [ ]:
vacancy_ids = vacancies_df["id"].values
result = []
i = 0
problem_ids = []
while i != len(vacancy_ids):
    response = requests.get(f"https://api.hh.ru/vacancies/{vacancy_ids[i]}")
    if response.status_code == 200:
        data = response.json()
        segmented_description = extract_requirements_segment(data["description"]).lower()
        for skill in data["key_skills"]:
            if len(skill) <= 45 and len(skill.split()) <= 3:
                skills.add(skill.lower())
        result.append((vacancy_ids[i], segmented_description))
    elif response.status_code == 404:
        problem_ids.append(id)
    else:
        print(f"Captcha at {i}'s request. Sleeping 10 seconds")
        time.sleep(10)
        continue
    if i % 100 == 0:
        print(i, "Done")
    i += 1

In [14]:
# Добавляем новые скилы при необходимости
new_skills = parsing_skills.difference(skills)
if len(new_skills) != 0:
    print("creating sql file")
    with open("add_new_skills.sql", 'w', encoding="utf-8") as file:
        file.write("insert into skills values\n")
        file.write(",\n".join([f"('{skill}')" for skill in new_skills]) + ";")
else:
    print("there is not new skills")

there is not new skills


In [28]:
skills_to_save = []
for description, skills in result:
    skills_to_save.append({"description": description, "skills": skills})
to_save = {"items": skills_to_save}
with open("json_results/skills_data.json", 'w', encoding='utf-16') as file:
    json.dump(to_save, file)

In [10]:
file = open("json_results/skills_data.json", 'r', encoding='utf-16')
vacancy_descriptions = json.loads(file.read())["items"]
file.close()

In [11]:
model = api.load("fasttext-wiki-news-subwords-300")

In [12]:
stop_words_en = set(stopwords.words('english'))
stop_words_ru = set(stopwords.words('russian'))
punctuation = set(string.punctuation)

skills_in_model = [s for s in tech_skills if s.lower() in model]

all_stop_words = stop_words_en | stop_words_ru | punctuation

In [13]:
def get_vacancy_skills(vacancies_id, text, skills_in_model):
    vacancy = extracted.lower()
    
    tokens = [t for t in word_tokenize(vacancy) if t in model]
    
    tokens = [t for t in tokens if t.lower() not in all_stop_words and t.strip() != '']

    token_matrix = np.array([model[t] for t in tokens])
    skill_matrix = np.array([model[s.lower()] for s in skills_in_model])

    similarities = cosine_similarity(token_matrix, skill_matrix)

    found_skills = {skills_in_model[j]
                    for i in range(len(tokens))
                    for j in range(len(skills_in_model))
                    if similarities[i, j] > 0.85}
    
    return [(vacancies_id, tech_skills[skill]) for skill in found_skills]

In [ ]:
vacancy_skills = []
for description in vacancy_descriptions:
    vacancy_skills += get_vacancy_skills(description[0], description[1], skills_in_model)